In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit

from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [41]:
doc = pd.read_csv('dataset/modified.txt',sep=',')
doc.head()

Sentence Final Intent
0              तिम्रो नाम के हो          परि
1  तिमिलाई के भनेर चिन्छन अरुले          परि
2     तिमिलाई बोलाउनि नाम के हो          परि
3       म तिमिलाई के भनेर बोलाउ          परि
4        तिमिलाई के भनेर चिन्छन          परि

In [77]:
df_with_res = pd.read_csv('dataset/follow_up_qsn_v1_1_new.csv',sep=',')

In [78]:
df_with_res.head()

Intent  Count                                           response
0           कोक    102  यहाँ कोक पाइन्छ। हामी सँग जम्बो कोक र सानो सिस...
1    चिकेन ससेज     90  यहाँ चिकेन ससेज पाइन्छ। चिकेन ससेज लाई प्लेटको...
2  चिकेन लेगपीस     90  यहाँ चिकेन ललिपप पाइन्छ। चिकेन ललिपप लाई प्लेट...
3     बफ सुकुटी     90  यहाँ बफ सुकुटी पाइन्छ।बफ सुकुटीलाई प्लेटको दुइ...
4   चिकेन ललिपप     90  यहाँ चिकेन ललिपप पाइन्छ। चिकेन ललिपप लाई प्लेट...

In [44]:
df_with_res.shape

(48, 2)

In [45]:
tgt = doc['Final Intent']

In [46]:
np.unique(tgt)


array(['अफर', 'अर्ना', 'आलु पकौडा', 'कम्पनि', 'कोक', 'खुकुरी', 'चाउमिन',
       'चिकेन चाउमिन', 'चिकेन चिल्ली', 'चिकेन पिजा', 'चिकेन मोमो',
       'चिकेन ललिपप', 'चिकेन लेगपीस', 'चिकेन ससेज', 'चिकेन सुकुटी',
       'चिज पिजा', 'चिसो', 'टुबोर्ग गोल्ड', 'टुबोर्ग डिलक्स',
       'टुबोर्ग स्ट्रोङ', 'डिउ', 'पकौडा', 'परि', 'पिजा', 'प्याज पकौडा',
       'फेन्टा', 'बफ चाउमिन', 'बफ चिल्ली', 'बफ मोमो', 'बफ ससेज',
       'बफ सुकुटी', 'बर्गर', 'बियर', 'भेज चाउमिन', 'भेज मोमो',
       'मसरूम पिजा', 'मिक्स पिजा', 'मोमो', 'रुसलान', 'रेस्टुरेन्ट',
       'वाईन', 'वाईफाई', 'ससेज', 'सी मोमो', 'सुकुटी', 'स्पेशल',
       'स्प्राइट', 'हुक्का'], dtype=object)

In [47]:
t2 = doc.copy()

In [48]:
sss = StratifiedShuffleSplit(test_size=0.1, random_state=42, n_splits=1)

In [49]:
for train_index, test_index in sss.split(t2,tgt):
    x_train, x_test = t2[t2.index.isin(train_index)], t2[t2.index.isin(test_index)]
    y_train, y_test = t2.loc[t2.index.isin(train_index),"Final Intent"], t2.loc[t2.index.isin(test_index),"Final Intent"]

In [50]:
x_train.shape

(2448, 2)

In [51]:
y_train.shape

(2448,)

In [52]:
x_test.shape

(273, 2)

In [53]:
y_test.shape

(273,)

In [54]:
def get_max_len(list1):
    len_list = [len(i) for i in list1]
    return max(len_list)

In [55]:
tokenizer = Tokenizer()

In [56]:
def conv_str_cols(col_tr,col_te):
    tokenizer = Tokenizer(num_words=1000)
    tokenizer.fit_on_texts(col_tr)
    col_tr1 = tokenizer.texts_to_sequences(col_tr)
    col_te1 = tokenizer.texts_to_sequences(col_te)
    max_len1 = get_max_len(col_tr1)
    col_tr2 = pad_sequences(col_tr1, maxlen=max_len1, dtype='int32',padding='post')
    col_te2 = pad_sequences(col_te1, maxlen=max_len1, dtype='int32',padding='post')
    
    return col_tr2,col_te2,tokenizer,max_len1
    

In [57]:
tr_padded,te_padded, tokenizer,max_len1 = conv_str_cols(x_train["Sentence"], x_test["Sentence"])

In [58]:
tr_padded.shape,te_padded.shape

# 11 is the length of sentence after padding and the former are the number of samples

((2448, 11), (273, 11))

In [59]:
max_len1

11

In [60]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [61]:
le = LabelEncoder()

In [62]:
y_train1 = le.fit_transform(y_train)
y_test1 = le.fit_transform(y_test)
y_train2 = to_categorical(y_train1)
y_test2 = to_categorical(y_test1)

In [63]:
classes_num = len(y_train.value_counts())

In [64]:
classes_num

48

In [65]:
## You embed the input layers and pass them to LSTM. The output of LSTM gets fed to a time-distributed dense layer. This layer gets fed into a 
# fully connected dense layer with classes_num as the number of nodes.

In [66]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import LSTM,Flatten,TimeDistributed

In [69]:
model = Sequential()
model.add(Embedding(1000, 100, input_length=max_len1))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(50, activation='relu')))
model.add(Flatten())
model.add(Dense(classes_num, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 11, 100)           100000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 11, 100)           80400     
_________________________________________________________________
lstm_7 (LSTM)                (None, 11, 50)            30200     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 11, 50)            2550      
_________________________________________________________________
flatten_3 (Flatten)          (None, 550)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 48)                26448     
Total params: 239,598
Trainable params: 239,598
Non-trainable params: 0
________________________________________________

In [70]:
model.fit(tr_padded, y_train2, epochs=10, verbose=2,batch_size=30)

Epoch 1/10
82/82 - 1s - loss: 3.5494 - accuracy: 0.0543
Epoch 2/10
82/82 - 1s - loss: 2.0726 - accuracy: 0.3321
Epoch 3/10
82/82 - 1s - loss: 1.1398 - accuracy: 0.5952
Epoch 4/10
82/82 - 1s - loss: 0.6928 - accuracy: 0.7680
Epoch 5/10
82/82 - 1s - loss: 0.4064 - accuracy: 0.8766
Epoch 6/10
82/82 - 1s - loss: 0.2654 - accuracy: 0.9240
Epoch 7/10
82/82 - 1s - loss: 0.1954 - accuracy: 0.9436
Epoch 8/10
82/82 - 1s - loss: 0.1712 - accuracy: 0.9530
Epoch 9/10
82/82 - 2s - loss: 0.0833 - accuracy: 0.9771
Epoch 10/10
82/82 - 1s - loss: 0.0390 - accuracy: 0.9910


### Model Prediction and Evaluation

In [71]:
pred_mat = model.predict_classes(te_padded)

In [72]:
from sklearn.metrics import accuracy_score


print (accuracy_score(y_test1, pred_mat))

0.9633699633699634


In [73]:
from sklearn.metrics import f1_score

print (f1_score(y_test1, pred_mat,average='micro'))
print (f1_score(y_test1, pred_mat,average='macro'))

0.9633699633699634
0.9562028271006598


In [79]:
def pred_new_text(txt1):
    print ("customer_text:",txt1)
    newdf = pd.DataFrame([txt1])
    newdf.columns = ["Sentence"]
    col_te1 = tokenizer.texts_to_sequences(newdf["Sentence"])
    col_te2 = pad_sequences(col_te1, maxlen=max_len1, dtype='int32',padding='post')
    class_pred = le.inverse_transform(model.predict_classes(col_te2))[0]
    resp = df_with_res.loc[df_with_res.Intent==class_pred,"response"].values[0]
    print ("Bot Response:",resp,"\n")
    return

In [80]:
pred_new_text("के स्पेशल छ")

customer_text: के स्पेशल छ
Bot Response: यो रेस्टुरेन्टमा स्पेशल पाउनि भनेको चिकेन सुकुटी चिकेन ललिपप र बफ चिल्ली हो | 



### Saving the Model

In [81]:
model.save("my_model")
model.save_weights("weights.h5")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model/assets


In [82]:
my_model = keras.models.load_model('my_model')
my_model.load_weights("weights.h5")

In [83]:
def pred_new_text(txt1):
    print ("customer_text:",txt1)
    newdf = pd.DataFrame([txt1])
    newdf.columns = ["Sentence"]
    col_te1 = tokenizer.texts_to_sequences(newdf["Sentence"])
    col_te2 = pad_sequences(col_te1, maxlen=max_len1, dtype='int32',padding='post')
    class_pred = le.inverse_transform(my_model.predict_classes(col_te2))[0]
    resp = df_with_res.loc[df_with_res.Intent==class_pred,"response"].values[0]
    print ("Bot Response:",resp,"\n")
    return

In [84]:
pred_new_text("मौसम कस्तो छ")

customer_text: मौसम कस्तो छ
Bot Response: यहाँ मोमो पाइन्छ र हामी सँग चिकेन मोमो बफ मोमो भेज मोमो र सी मोमो  छ| मोमो बन्न पन्ध्र मिनेट लाग्छ| 



### Speech Recognition

In [65]:
! pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 5.0 MB/s eta 0:00:01     |██████████▌                     | 10.8 MB 5.1 MB/s eta 0:00:05
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [86]:
import speech_recognition as sr
import webbrowser

r1 = sr.Recognizer()
r2 = sr.Recognizer()
r3 = sr.Recognizer()

In [87]:
def to_audio():
    with sr.Microphone() as source:
        print('Speak Now')
        audio = r3.listen(source)
    return audio

In [99]:
def speech_to_text(af,show_all):
    abc = r.recognize_google(af,show_all=show_all,language='ne-NP')
    return abc

In [115]:
def translate():
    af1= to_audio()
    x = speech_to_text(af1,show_all=True)
    return x['alternative'][0]['transcript']

In [117]:
def final_prediction():
    text = translate()
    return pred_new_text(text)

In [162]:
final_prediction()

Speak Now
customer_text: mushroom pizza छ
Bot Response: यहाँ मसरुम पिजा पाइन्छ| स्माल साइजको तिन सय मिडियम साइजको चार सय र लार्ज साइजको पाँच सय| 

